# Coursera Capstone - Battle of the Neighborhoods


## Introduction
This is the final project for the IBM Data Science Professional Certificate. I will be using Foursquare location data to explore neighborhoods in Milwaukee, WI.
## Background
A company notices the high growth rates of the number of young people in Milwaukee, WI, and wants to capitalize on it. They want to open a new restaurant in the city that will attract young people by providing a fun place to eat and hang out with their friends.
## Problem
The company is unfamiliar with the area and they need to choose the best location for the new restaurant so that it gets a lot of foot traffic and is in a convenient place for customers to get to. They have contracted our firm to find the best place for this new hang out.

 



## Data
For this analysis I will be using two data sets. The first data set is a list of all the neighborhoods in Milwaukee and their respective coordinates. This list also includes the district that the neighborhood is in. I used Wikipedia and Google to manually create this CSV file. Although the majority of neighborhoods had readily available coordinates, I had to base a few of them off locations that were in the neighborhood.

The second data source I will be using is venue data from the Foursquare API. I will use the neighborhood coordinates to query the API to get data about the venues near each location. 

![Neighborhoods](mke.png)
 

